###Домашняя работа 1. Базовая работа с прочтениями.
##Задание
Загрузите прочтения всех экспериментов из статьи [Evaluation of two main RNA-seq approaches for gene quantification in clinical RNA sequencing: polyA+ selection versus rRNA depletion](https://www.nature.com/articles/s41598-018-23226-4), в которой сравнивают два протокола подготовки библиотек для bulk RNA-Seq: при помощи деплеции рРНК (=total) или при помощи селекции polyA-транскриптов (=polyA+).

При выполнении задания используйте Nextflow pipeline.

Загружались два прочтения:
*  SRR6410611 SRX3503729: mRNA-seq of blood <br/>
number of Spots: 49,997,942 <br/>
nubmer of Bases: 7.5G <br/>
Size: 3Gb <br/>
Published: 2017-12-21 <br/> 


* SRR6410612 SRX3503728: mRNA-seq of blood<br/>
number of Spots: 49,997,356<br/>
number of Bases: 7.5G <br/>
Size:  3Gb<br/>
Published:2017-12-21  

In [ ]:
!./nextflow run hw1_pipeline.nf -with-report nextflow_report.html -with-dag flowchart.pdf --file SRA.txt --results_dir homework_1

Файл SRA.txt
```
SRR6410611
SRR6410612
```

Скрипт hw1_pipeline.nf
```
params.results_dir = "results/"
SRA_file = new File(params.file).readLines()

log.info "List of files: ${SRA_file}"
log.info ""
log.info "  Q U A L I T Y   C O N T R O L  "
log.info "================================="
log.info "SRA number         : ${SRA_file}"
log.info "Results location   : ${params.results_dir}"

process DownloadFastQ {
  publishDir "${params.results_dir}"

  input:
    val sra

  output:
    path "${sra}/*"

  script:
    """
    /content/sratoolkit.3.0.0-ubuntu64/bin/fastq-dump --gzip --split-3 ${sra} -O ${sra}/
    """
}

process QC {
  input:
    path x

  output:
    path "qc/*"

  script:
    """
    mkdir qc
    /content/FastQC/fastqc -o qc $x
    """
}

process MultiQC {
  publishDir "${params.results_dir}"

  input:
    path x

  output:
    path "multiqc_report.html"

  script:
    """
    multiqc $x
    """
}

workflow {
  data = Channel.fromList( SRA_file )
  DownloadFastQ(data)
  QC( DownloadFastQ.out )
  MultiQC( QC.out.collect() )
}
```